# **Reconocimiento Facial de Colaboradores usando Red Convolucional Siamesa**

### Proyecto Final de Técnicas Avanzadas de Data Mining y Sistemas Inteligentes
Este proyecto permite el reconocimiento de rostros en tiempo real mediante el uso de una camara web.

La solución continene los siguientes pasos principales:
  - Detección de rostro.
  - Reconocimiento de rostro.
  
Para el paso de detección de rostro nos apoyamos de la funcion de **OpenCV "Haar Cascade"**, posteriormente obtenemos el bounding box y ajustamos la imágen para que pueda ingresar al modelo. Para el paso de Reconocimiento de rostro de utilizó el **modelo pre-entrenado FaceNet** el cual nos brinda buenos vectores caracteristicos debido a que usa la **función de perdida en tripleta (triplet loss function)**.
  
## Autores
* **Carlo Cano Gordillo**
* **Oscar Moreno Feliz**
* **Carlos Roca Bejar**

## 1. Importación de librerías

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
from keras.models import load_model
K.set_image_data_format('channels_first')

import pickle
from PIL import Image
import cv2
import os.path
import os
import time
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf

Using TensorFlow backend.


In [2]:
import keras.backend.tensorflow_backend as tfback

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus

## 1. Detección de rostro

#### 1.1. Función Haarcascades por medio de camara web

In [39]:
face_cascade = cv2.CascadeClassifier(
        r'haarcascades/haarcascade_frontalface_default.xml')

In [28]:
def detect_face():
    save_loc = r''+folder_web_camera_image+'/temporal.jpg'
    save_loc_cut_image = r''+folder_web_camera_image+'/temporal_cut.jpg'
    capture_obj = cv2.VideoCapture(0)
    # Establecemos el tamaño del marco de la cámara web
    capture_obj.set(3, 640)
    capture_obj.set(4, 480)
    
    if capture_obj.isOpened():
        face_found = False
        req_sec = 3
        loop_start = time.time()
        elapsed = 0

        while(True):
            curr_time = time.time()
            elapsed = curr_time - loop_start # Contador para un máximo de 3 segundos para que detecte el rostro de la persona
            if elapsed >= req_sec:
                break

            ret, frame = capture_obj.read()
            frame = cv2.flip(frame, 1, 0)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5) # Se detectan los rostros en el frame actual
            if len(faces) > 1:
                print("¡ERROR!, Sólo debe haber una persona en la foto.")
            else:
                x, y, w, h = faces[0] # se obtiene las coordenadas del rostro de la persona
                roi_color = frame[y:y+h, x:x+w]
                cv2.imwrite(save_loc, roi_color) # recortamos la imagen del frame para obtener solo el rostro y lo almacenamos
                cv2.rectangle(frame, (x-10, y-70), (x+w+20, y+h+40), (15, 175, 61), 4) # dibujamos el bounding box para que se muestre en la cámara web 

            cv2.imshow('frame', frame)
            # para salir de la cámara web presionar la tecla 'q'
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        capture_obj.release()
        cv2.destroyAllWindows()
    else:
        print("No se puede acceder a la cámara.")

    img = cv2.imread(save_loc)
    if img is not None:
        face_found = True
    else:
        face_found = False

    return face_found

#### 1.2 Obtener vector característico de una imágen

In [4]:
def img_to_encoding(image_path, model):
    img1 = cv2.imread(image_path, 1)
    img = img1[...,::-1]
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

#### 1.3. Funciones de procesamiento de imágen

In [5]:
folder_web_camera_image = 'saved_image'

In [6]:
def ini_user_database():
    if os.path.exists('database/user_dict.pickle'):
        with open('database/user_dict.pickle', 'rb') as handle:
            user_db = pickle.load(handle)   
    else:
        user_db = {}
        os.makedirs('database')
        with open('database/user_dict.pickle', 'wb') as handle:
            pickle.dump(user_db, handle, protocol=pickle.HIGHEST_PROTOCOL)   
    return user_db

In [7]:
def add_user_img_path(user_db, model, name, img_path):
    if name not in user_db: 
        user_db[name] = img_to_encoding(img_path, model)
        with open('database/user_dict.pickle', 'wb') as handle:
                pickle.dump(user_db, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print('Usuario ' + name + ' agregado.')
    else:
        print('El usuario "' + name + '" ya ha sido registrado.')

In [8]:
def resize_img(image_path):
    img = cv2.imread(image_path, 1)
    img = cv2.resize(img, (160, 160))
    cv2.imwrite(image_path, img)

In [9]:
def add_user_webcam(user_db, FRmodel, name):
    if name not in user_db: 
        face_found = detect_face()
        if face_found:
            temp_img = folder_web_camera_image + "/temporal.jpg"
            resize_img(temp_img)
            add_user_img_path(user_db, FRmodel, name, temp_img)
        else:
            print('No se ha encontrado un rostro en el frame.')
    else:
        print('El usuario "' + name + '" ya ha sido registrado.')

In [10]:
def delete_user(user_db, name):
    popped = user_db.pop(name, None)
    
    if popped is not None:
        print('Usuario ' + name + ' eliminado.')
        with open('database/user_dict.pickle', 'wb') as handle:
                pickle.dump(user_db, handle, protocol=pickle.HIGHEST_PROTOCOL)
    elif popped == None:
        print('Usuario no encontrado.')

## 2. Reconocimiento de rostro

In [11]:
def find_face(image_path, database, model, threshold=0.6):
    encoding = img_to_encoding(image_path, model)

    min_dist = 99999
    for name in database:
        dist = np.linalg.norm(np.subtract(database[name], encoding))
        print("Nombre: " + name + ", distancia: " + str(dist))
        if dist < min_dist:
            min_dist = dist
            identity = name

    if min_dist > threshold:
        print("Usuario no ha sido identificado.")
        identity = 'Persona desconocida'
    else:
        print("¡Hola " + str(identity) + "!")

    return min_dist, identity # Retornamos la persona con menos distancia en base a su vector caracteristico

In [12]:
def do_face_recognition_from_image(user_db, model, threshold, img_loc):
    img = cv2.imread(img_loc, 1)
    img = cv2.resize(img, (160, 160))
    cv2.imwrite(img_loc, img)

    find_face(img_loc, user_db, model, threshold)

## 3. Importación de modelo
Previamente se debe descargar el modelo [facnet_keras.h5](https://drive.google.com/file/d/1wsJs5ZnhI7meqdOX6S9Indm9l1zmsisH/view?usp=sharing) y guardarlo dentro de la carpeta "models".

#### Función de perdida de tripleta
El objetivo de esta función de perdida es disminuir la distancia entre instancias de una misma clase y alejar aquellas de clases diferentes.

![triplet_loss_function](assets/triplet_loss_function.png)

In [13]:
def triplet_loss(y_pred, alpha = 0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    pos_dist = tf.reduce_sum( tf.square(tf.subtract(y_pred[0], y_pred[1])) )
    neg_dist = tf.reduce_sum( tf.square(tf.subtract(y_pred[0], y_pred[2])) )
    basic_loss = pos_dist - neg_dist + alpha
    
    loss = tf.maximum(basic_loss, 0.0)
   
    return loss

In [14]:
FRmodel = load_model('models/facenet_keras.h5', custom_objects={'triplet_loss': triplet_loss})

/home/carlos/anaconda3/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


# **Uso del sistema**

In [15]:
user_db = ini_user_database()

### 1. Agregar nuevo usuario

In [33]:
add_user_webcam(user_db, FRmodel, "Luciano Leon")

Usuario Luciano Leon agregado.


##### 2. Eliminar usuario (Opcional)

In [25]:
delete_user(user_db, "Carlos")

Usuario Carlos eliminado.


### 3. Reconocimiento facial de colaborador

In [36]:
def face_identification(user_db, model):
    face_found = detect_face()
    if face_found:
        do_face_recognition_from_image(user_db, model, 7, folder_web_camera_image + "/temporal.jpg")
    else:
        print('No se ha encontrado un rostro en el frame.')
    

In [37]:
face_identification(user_db, FRmodel)

Nombre: Carlos Perez, distancia: 10.844667
Nombre: Ricardo Bonifaz, distancia: 8.139328
Nombre: Sergio Gonzales, distancia: 7.3957644
Nombre: Franco Prado, distancia: 11.089435
Nombre: Fiorela Quintanilla, distancia: 15.384703
Nombre: Jesus Echegaray, distancia: 6.9111333
Nombre: Cliff Zurita, distancia: 9.04925
Nombre: Giancarlo Diaz, distancia: 13.694735
Nombre: Luciano Leon, distancia: 6.676095
¡Hola Luciano Leon!


##### 4. Agregar usuarios de manera masiva
Para esta funcionalidad se debe agregar una carpeta con el nombre de la persona dentro de la carpeta "people". Dentro de la carpeta con el nombre de la persona agregar la imágen de tamaño mayor a 500x500.

In [40]:
people_path = 'people/'

In [41]:
for name in os.listdir(people_path):
    name_path = os.path.join(people_path, name)
    for filename in os.listdir(name_path):
        image_path = os.path.join(name_path, filename)
        image = cv2.imread(image_path, 1)
        
        frame = cv2.flip(image, 1, 0)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        if len(faces) > 1:
            print("¡ERROR!, Sólo debe haber una persona en la foto.")
        else:
            x, y, w, h = faces[0]
            roi_color = frame[y:y+h, x:x+w]
            resize_img = cv2.resize(roi_color, (160, 160))
            temp_img = folder_web_camera_image + "/temporal.jpg"
            cv2.imwrite(temp_img, resize_img)
            add_user_img_path(user_db, FRmodel, name, temp_img)

El usuario "Carlos Perez" ya ha sido registrado.
El usuario "Ricardo Bonifaz" ya ha sido registrado.
El usuario "Sergio Gonzales" ya ha sido registrado.
El usuario "Franco Prado" ya ha sido registrado.
El usuario "Fiorela Quintanilla" ya ha sido registrado.
El usuario "Jesus Echegaray" ya ha sido registrado.
El usuario "Cliff Zurita" ya ha sido registrado.
¡ERROR!, Sólo debe haber una persona en la foto.
El usuario "Giancarlo Diaz" ya ha sido registrado.


# **Reconocimiento facial en tiempo real**

#### 1. Reconocimiento facil en tiempo real

In [18]:
def find_face_realtime(image_path, database, model, threshold):
    encoding = img_to_encoding(image_path, model)
    registered = False
    min_dist = 99999
    identity = 'Persona desconocida'
    for name in database:
        dist = np.linalg.norm(np.subtract(database[name], encoding))
        print("Nombre: " + name + ", distancia: " + str(dist))
        if dist < min_dist:
            min_dist = dist
            identity = name

    if min_dist > threshold:
        registered = False
    else:
        registered = True
    return min_dist, identity, registered


#### 2. Detección de rostro en tiempo real

In [19]:
def detect_face_realtime(database, model, threshold=6):
    text = ''
    font = cv2.FONT_HERSHEY_SIMPLEX
    save_loc = r''+folder_web_camera_image+'/temporal.jpg'
    capture_obj = cv2.VideoCapture(0)
    capture_obj.set(3, 640)
    capture_obj.set(4, 480)
    prev_time = time.time()
    while(True):
        ret, frame = capture_obj.read()
        frame = cv2.flip(frame, 1, 0)

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            roi_color = frame[y-90:y+h+70, x-50:x+w+50]
            cv2.imwrite(save_loc, roi_color)
            curr_time = time.time()

            if curr_time - prev_time >= 3:
                img = cv2.imread(save_loc)
                if img is not None:
                    resize_img(save_loc)

                    min_dist, identity, registered = find_face_realtime(
                        save_loc, database, model, threshold)

                    if min_dist <= threshold and registered:
                        text = 'Bienvenido ' + identity
                        print(text)
                    else:
                        text = 'Usuario desconocido'
                        print('¡Usuario desconocido detectado!')
                    print('Distancia minima:' + str(min_dist))
                    print("--------------------------------------------")
                prev_time = time.time()

            cv2.rectangle(frame, (x-10, y-70),
                          (x+w+20, y+h+40), (15, 175, 61), 4)
            cv2.putText(frame, text, (50, 50), font, 1, (158, 11, 40), 3)

        cv2.imshow('frame', frame)

        # close the webcam when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    capture_obj.release()
    cv2.destroyAllWindows()

#### 3. Ejecución de funciones
Con el parámetro "threshold" se puede regular la distancia entre vectores caracteristicos.

In [20]:
detect_face_realtime(user_db, FRmodel, threshold = 6)

Nombre: Jesus, distancia: 8.0548935
Nombre: Anel, distancia: 8.008589
Nombre: Daniel, distancia: 9.617259
Nombre: Gonzalo, distancia: 8.265307
Nombre: Carlos, distancia: 4.2157307
Nombre: Carlos Perez, distancia: 7.8946266
Nombre: Ricardo Bonifaz, distancia: 6.012129
Nombre: Sergio Gonzales, distancia: 6.5055523
Nombre: Franco Prado, distancia: 7.8562565
Nombre: Fiorela Quintanilla, distancia: 10.19094
Nombre: Jesus Echegaray, distancia: 7.2403016
Nombre: Cliff Zurita, distancia: 7.013948
Nombre: Giancarlo Diaz, distancia: 8.403421
Bienvenido Carlos
Distancia minima:4.2157307
--------------------------------------------
Nombre: Jesus, distancia: 8.109145
Nombre: Anel, distancia: 7.4207654
Nombre: Daniel, distancia: 9.42279
Nombre: Gonzalo, distancia: 7.971423
Nombre: Carlos, distancia: 3.374651
Nombre: Carlos Perez, distancia: 7.7983418
Nombre: Ricardo Bonifaz, distancia: 6.090741
Nombre: Sergio Gonzales, distancia: 6.380297
Nombre: Franco Prado, distancia: 7.802237
Nombre: Fiorela Qui